# Ibrahim Gabr - 12144496

In [1]:
import warnings
warnings.filterwarnings(action='ignore',category=FutureWarning)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

%run preprocessing.py # pre-processing function, uses helper functions

Using TensorFlow backend.


# Section 1

Here are my Sentences:

< s > Ibrahim eats < /s>

< s > Ibrahim < /s>

< s > Ibrahim < /s>

< s > Ibrahim < /s>

< s > Ibrahim < /s>

< s > Ibrahim < /s>

**Model U:**

**Bigram, count, probability:**

< s > Ibrahim, 6, 0.5

Ibrahim < /s>, 5, 0.417

< s > eats, 0, 0

eats < /s>, 1, 0.083

**Model S:**

**Bigram, count, probability:**

< s > Ibrahim, 7, 0.4375

Ibrahim < /s>, 6, 0.375

< s > eats, 1, 0.0625

eats < /s>, 2, 0.125

**Results:**

**sentence: < s > Ibrahim eats < /s>**

Under Model U, the probability of this sentence is: $0.5 \cdot 0.083 = 0.0415$

Under Model S, the probability of this sentence is: $0.4375 \cdot 0.125 = 0.0546875$

As such, this sentence has a **higher probability under Model S than Model U.**




## Pre-processing Stage

In [2]:
training_tweets = open("tweets-train.txt", "r").readlines()
devtest_tweets = open("tweets-devtest.txt", "r").readlines()
dev_tweets = open("tweets-dev.txt", "r").readlines()
embeddings = open("embeddings-twitter.txt", "r").readlines()

In [3]:
training_input, training_target = pre_process_file(training_tweets, training_tweets, embeddings)

In [4]:
dev_input, dev_target = pre_process_file(training_tweets, dev_tweets, embeddings)

In [5]:
dev_test_input, dev_test_target = pre_process_file(training_tweets, devtest_tweets, embeddings)

# Section 2.1

Accuracy on Dev Test is **85.6%**. Code is below

In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(1,150), activation="tanh")) # issue is here.
model.add(Dense(25, activation='softmax'))

model.compile(loss='categorical_crossentropy', # Cross-entropy
                optimizer="adam",#using adam optimizer
                metrics=['accuracy']) # Accuracy performance metric

callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best.h5', monitor='val_loss', save_best_only=True)]

history = model.fit(training_input,
                      training_target,
                      epochs=100,
                      callbacks=callbacks,
                      batch_size=50,
                      validation_data=(dev_input, dev_target))


Train on 14619 samples, validate on 4823 samples
Epoch 1/100
14619/14619 [==============================] - 1s 56us/step - loss: 1.3758 - acc: 0.6585 - val_loss: 0.7945 - val_acc: 0.7871
Epoch 2/100
14619/14619 [==============================] - 1s 44us/step - loss: 0.6481 - acc: 0.8206 - val_loss: 0.6031 - val_acc: 0.8350
Epoch 3/100
14619/14619 [==============================] - 1s 45us/step - loss: 0.5356 - acc: 0.8489 - val_loss: 0.5483 - val_acc: 0.8453
Epoch 4/100
14619/14619 [==============================] - 1s 46us/step - loss: 0.4866 - acc: 0.8607 - val_loss: 0.5241 - val_acc: 0.8488
Epoch 5/100
14619/14619 [==============================] - 1s 51us/step - loss: 0.4588 - acc: 0.8676 - val_loss: 0.5134 - val_acc: 0.8503
Epoch 6/100
14619/14619 [==============================] - 1s 46us/step - loss: 0.4393 - acc: 0.8719 - val_loss: 0.5060 - val_acc: 0.8530
Epoch 7/100
14619/14619 [==============================] - 1s 45us/step - loss: 0.4249 - acc: 0.8743 - val_loss: 0.5010 - v

In [10]:
best_model = load_model("best.h5")

In [11]:
best_model.evaluate(dev_test_input, dev_test_target, batch_size=50)

7152/7152 [==============================] - 0s 14us/step


[0.4807981384347216, 0.8561241612071693]

## Section 2.2

In this section, I add 3 layers, all with the relu activation function in addition to 50% dropout between every layer.

This results in an accuracy of ~ 87.4%.

This is around a 2% increase from the baseline architecture.

The literature suggests that relu is the best acivation function to use, this is why I chose it as the activation.

Furthermore, while early stopping is itself a form of regularization, adding dropout to "further" regularize the model seems to have improved performance.

**Experimentation Details:**

I experimented with higher dropout (0.7) and adding a fourth layer of 4096 nodes, and this did not imporve network performance.

I also experimented with changing the activation used in different layers, for example: Relu -> tanh -> relu -> sigmoid. This did not result in any performance gains.

I tried to research whether there are any benefits of altering the activation fucntion on a layer by layer basis and could find no conclusive evidence that it does. As such, each layer just employs relu as the activation function.

The adam optimizer outpeformed SGD. Furthermore, my early stopping criterion was 2, that is, after seeing no improvement in validation loss for 2 epochs, stop training the network. Initially, I set this to 5 in the baseline model, however, reducing the `patience` led to better model performance.

Lastly, there does seem to be some tradeoff between the **depth** and the **width** of the model. I have found that having a wider model (i.e more nodes in a given layer) gives better performance off the bat, whereas having a deeping network could lead to marginal performance gains (i.e. Accuracy).

In [31]:
model = Sequential()
model.add(Dense(512, input_shape=(1,150), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2048, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(25, activation='softmax'))

model.compile(loss='categorical_crossentropy', # Cross-entropy
                optimizer="adam",#using adam optimizer
                metrics=['accuracy']) # Accuracy performance metric

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_2_layers.h5', monitor='val_loss', save_best_only=True)]

history = model.fit(training_input,
                      training_target,
                      epochs=100,
                      callbacks=callbacks,
                      batch_size=50,
                      validation_data=(dev_input, dev_target))


Train on 14619 samples, validate on 4823 samples
Epoch 1/100
14619/14619 [==============================] - 16s 1ms/step - loss: 1.0919 - acc: 0.6856 - val_loss: 0.5893 - val_acc: 0.8325
Epoch 2/100
14619/14619 [==============================] - 15s 998us/step - loss: 0.6083 - acc: 0.8195 - val_loss: 0.5121 - val_acc: 0.8553
Epoch 3/100
14619/14619 [==============================] - 14s 982us/step - loss: 0.5104 - acc: 0.8466 - val_loss: 0.4858 - val_acc: 0.8627
Epoch 4/100
14619/14619 [==============================] - 15s 995us/step - loss: 0.4604 - acc: 0.8626 - val_loss: 0.4742 - val_acc: 0.8642
Epoch 5/100
14619/14619 [==============================] - 14s 946us/step - loss: 0.4166 - acc: 0.8753 - val_loss: 0.4655 - val_acc: 0.8748
Epoch 6/100
14619/14619 [==============================] - 15s 1ms/step - loss: 0.3875 - acc: 0.8808 - val_loss: 0.4615 - val_acc: 0.8754
Epoch 7/100
14619/14619 [==============================] - 15s 1ms/step - loss: 0.3641 - acc: 0.8884 - val_loss: 0.

In [32]:
best_model = load_model("best_2_layers.h5")
best_model.evaluate(dev_test_input, dev_test_target, batch_size=50)

7152/7152 [==============================] - 1s 171us/step


[0.435658347042865, 0.8740212483460738]